In [7]:
import numpy as np
import pandas as pd

# Load the data from the uploaded files
agent_salary_data = pd.read_csv('[BADSS case] Agent Salary and Eligibility Threshold - data.csv')
existing_agent_data = pd.read_csv('[BADSS case] Existing Agent Staffing - data.csv')
advertiser_signups_data = pd.read_csv('simulated_advertiser_signups_2025_daily_with_ID.csv')

# Merge agent salary and existing agent staffing data on 'Country'
countries_data = pd.merge(agent_salary_data, existing_agent_data, on="Country")

# Simulate the arrival of advertisers (Poisson Process)
def simulate_advertiser_signups(lambda_rate, days=365):
    # Poisson process for advertiser sign-ups
    signups = np.random.poisson(lambda_rate, size=days)
    return signups

# Q-learning setup for agent management
class QLearningAgent:
    def __init__(self, actions, state_size, learning_rate=0.1, discount_factor=0.9, epsilon=0.1):
        self.actions = actions
        self.state_size = state_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.q_table = np.zeros((state_size, len(actions)))

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:  # Explore
            return np.random.choice(self.actions)
        else:  # Exploit
            return np.argmax(self.q_table[state])

    def learn(self, state, action, reward, next_state):
        best_future_q = np.max(self.q_table[next_state])
        self.q_table[state, action] += self.learning_rate * (reward + self.discount_factor * best_future_q - self.q_table[state, action])

# Simulate daily operations
def simulate_day(state, q_learning_agent, agent_capacity_per_agent, advertiser_signups, available_agents, agent_daily_wage, total_wage_cost_per_day, hired_agents, fired_agents):
    # Action space: 0 = keep same, 1 = hire agents, 2 = fire agents
    action = q_learning_agent.choose_action(state)

    # If action is to hire agents
    if action == 1:  # Hire up to 80 agents
        hire_count = np.random.randint(1, 81)  # Hire between 1 and 80 agents
        available_agents += hire_count
        hired_agents += hire_count
    # If action is to fire agents
    elif action == 2:  # Fire up to 80 agents
        fire_count = np.random.randint(1, 81)  # Fire between 1 and 80 agents
        available_agents = max(available_agents - fire_count, 0)  # Ensure we don't have negative agents
        fired_agents += fire_count

    # Check if advertisers are signing up today
    new_advertisers = advertiser_signups[state]
    print(f"Day {state+1}: Advertisers Signing Up = {new_advertisers}")  # Debugging line

    # Assign advertisers to available agents
    assigned_advertisers = min(new_advertisers, available_agents * agent_capacity_per_agent)
    print(f"Day {state+1}: Assigned Advertisers = {assigned_advertisers}")  # Debugging line

    # Calculate uplift revenue
    uplift_revenue = 0.0
    for _ in range(assigned_advertisers):
        uplift = np.random.choice([0, 5, 10, 15, 20, 25], p=[0.05, 0.15, 0.25, 0.25, 0.2, 0.1])
        uplift_revenue += 1000 * (uplift / 100.0)  # Assume each advertiser has a projected budget of 1000 USD
    print(f"Day {state+1}: Uplift Revenue = {uplift_revenue}")  # Debugging line

    # Total revenue (from uplift and advertiser budgets after 60 days)
    total_revenue = uplift_revenue

    # Calculate net revenue by subtracting the wage cost and fire penalty
    net_revenue = total_revenue - total_wage_cost_per_day

    # If agents have been fired, apply a 4-month wage penalty per agent
    fire_penalty = fired_agents * (4 * agent_daily_wage)
    net_revenue -= fire_penalty

    # Calculate reward as net revenue for this day
    reward = net_revenue

    # Learn from the current state-action pair
    q_learning_agent.learn(state, action, reward, state)  # Assume state remains the same for simplicity

    return available_agents, total_revenue, net_revenue, hired_agents, fired_agents

# Simulate for multiple countries
def simulate_for_multiple_countries(countries_data, advertiser_signups_data, advertiser_lambda_rate=5, days_in_year=365):
    all_results = []

    # Loop through each country
    for _, data in countries_data.iterrows():
        # Initialize Q-learning agent
        actions = [0, 1, 2]  # Action space (0: keep, 1: hire, 2: fire)
        state_size = days_in_year  # State space size (1 state per day for simplicity)
        q_learning_agent = QLearningAgent(actions, state_size)

        # Extract country-specific information
        agent_annual_salary = data['Annual_Agent_Salary_USD']
        agent_daily_wage = agent_annual_salary / 365
        existing_agents_count = data['Existing_Agent_Count']
        total_wage_cost_per_day = existing_agents_count * agent_daily_wage

        # Filter advertiser sign-ups data for the specific country
        country_signups = advertiser_signups_data[advertiser_signups_data['Country'] == data['Country']]
        advertiser_signups = country_signups.groupby('Sign_Up_Date').size().reindex(range(1, days_in_year+1), fill_value=0).values

        # Track revenue and net revenue
        total_revenue = 0
        net_revenue = 0
        available_agents = existing_agents_count
        daily_results = []

        # Track monthly agent numbers
        monthly_agents = []

        # Track hired and fired agents for penalties and ramp-up times
        hired_agents = 0
        fired_agents = 0

        # Simulate each day of the year
        for day in range(1, days_in_year + 1):
            # For simplicity, we assume that each day is its own state
            state = day - 1  # Day as state index (0-based)
            available_agents, daily_revenue, daily_net_revenue, hired_agents, fired_agents = simulate_day(
                state, q_learning_agent, data['Advertiser_Eligibility_Threshold_USD'], advertiser_signups,
                available_agents, agent_daily_wage, total_wage_cost_per_day, hired_agents, fired_agents)

            # Accumulate revenue
            total_revenue += daily_revenue
            net_revenue += daily_net_revenue

            # Store daily results
            daily_results.append({
                'Day': day,
                'Available_Agents': available_agents,
                'Daily_Revenue': daily_revenue,
                'Daily_Net_Revenue': daily_net_revenue
            })

            # Track the available agents at the end of each month (every 30 days)
            if day % 30 == 0:
                monthly_agents.append({
                    'Month': day // 30,
                    'Available_Agents': available_agents
                })

        # Add results for this country
        all_results.append({
            'Country': data['Country'],
            'Daily_Results': daily_results,
            'Monthly_Agents': monthly_agents,
            'Total_Revenue': total_revenue,
            'Total_Net_Revenue': net_revenue
        })

    return all_results


# Run simulation for multiple countries using the data loaded earlier
simulation_results = simulate_for_multiple_countries(countries_data, advertiser_signups_data)

# Output results for each country
for country_result in simulation_results:
    print(f"\nResults for {country_result['Country']}:")

    # Final Revenue and Net Revenue
    print(f"Total Revenue for the year: ${country_result['Total_Revenue']:.2f}")
    print(f"Total Net Revenue for the year: ${country_result['Total_Net_Revenue']:.2f}")

    # Print monthly agent counts
    print("\nMonthly Agent Counts:")
    monthly_agents_df = pd.DataFrame(country_result['Monthly_Agents'])
    print(monthly_agents_df.to_string(index=False))

    # Print daily results (optional, can be huge)
    daily_results_df = pd.DataFrame(country_result['Daily_Results'])
    print("\nDaily Revenue and Net Revenue:")
    print(daily_results_df[['Day', 'Available_Agents', 'Daily_Revenue', 'Daily_Net_Revenue']].to_string(index=False))


Streaming output truncated to the last 5000 lines.
  10               442            0.0      -23963.665753
  11               457            0.0      -23963.665753
  12               457            0.0      -23963.665753
  13               457            0.0      -23963.665753
  14               457            0.0      -23963.665753
  15               457            0.0      -23963.665753
  16               457            0.0      -23963.665753
  17               457            0.0      -23963.665753
  18               457            0.0      -23963.665753
  19               457            0.0      -23963.665753
  20               457            0.0      -23963.665753
  21               457            0.0      -23963.665753
  22               457            0.0      -23963.665753
  23               457            0.0      -23963.665753
  24               457            0.0      -23963.665753
  25               457            0.0      -23963.665753
  26               457            0.0